In [2]:
import torch

x = torch.tensor([[2,1],[5,6],[3,7],[12,0]], dtype=torch.float32)
y = torch.tensor([0,1,0,1], dtype=torch.float32)



边的定义无所谓

In [3]:
edge_index = torch.tensor([[0,1,2,0,3],   #起始点
                           [1,0,1,3,2]], dtype=torch.long) #终止点

创建torch_geometric的图

In [4]:
from torch_geometric.data import Data

data = Data(x=x, y=y, edge_index=edge_index)

In [5]:
data

Data(x=[4, 2], edge_index=[2, 5], y=[4])

1、节点：（汽车、地铁站、人物、分子）

2、边：（节点间是否连接、节点间的相似性）

3、节点特征（node feature）：（比如某个地铁站的人流量）

4、标签（label）：（这取决于你做什么类型的任务：节点、边、图级别的标签）

以上四点是一个标准图数据集必须包含的内容，除此外，如果你的数据集内容较为丰富，可能还包含以下内容：

5、边的权重（例如每条边的连接强度）

6、边的特征（例如包含不同类型的边）

第五点和第六点在RGCN等关系图卷积神经网络中具有应用。

同质图（Homogeneous）：你可以简单理解为你的图中的节点/边是同一种类型，例如交通网络中节点都是地铁站，没有公交车站

异质图（Heterogeneous）：存在不同类型的节点或者边的类型。例如交通网络中包含了地铁站、自行车站点、公交车站点。

同质图为例:

FIFA 21 Rating 数据集，一个包含足球运动员的数据集。

In [6]:
import requests
url = "https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player.csv"
filename = "tbl_player.csv"

response = requests.get(url)
response.raise_for_status()

with open(filename, "wb") as file:
    file.write(response.content)

print("文件下载完成！")

文件下载完成！


In [7]:
import pandas as pd
import requests
# Download data (quietly)
# !wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player.csv
# !wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player_skill.csv
# !wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_team.csv



# 加载数据
player_df = pd.read_csv("tbl_player.csv" )
skill_df = pd.read_csv("tbl_player_skill.csv")
team_df = pd.read_csv("tbl_team.csv")

# Extract subsets
player_df = player_df[["int_player_id", "str_player_name", "str_positions", "int_overall_rating", "int_team_id"]]
skill_df = skill_df[["int_player_id", "int_long_passing", "int_ball_control", "int_dribbling"]]
team_df = team_df[["int_team_id", "str_team_name", "int_overall"]]

# Merge data
player_df = player_df.merge(skill_df, on='int_player_id')
fifa_df = player_df.merge(team_df, on='int_team_id')

# 按照int_overall_rating排序
fifa_df = fifa_df.sort_values(by="int_overall_rating", ascending=False)
print("Players: ", fifa_df.shape[0])
fifa_df.head()

Players:  18767


,int_player_id,str_player_name,str_positions,int_overall_rating,int_team_id,int_long_passing,int_ball_control,int_dribbling,str_team_name,int_overall
0,1,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,5.0,91,96,96,FC Barcelona,84
1,2,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,6.0,77,92,88,Juventus,83
2,3,Jan Oblak,GK,91,8.0,40,30,12,Atlético Madrid,83
3,4,Kevin De Bruyne,"CAM, CM",91,2.0,93,92,88,Manchester City,85
4,5,Neymar da Silva Santos Júnior,"LW, CAM",91,7.0,81,95,95,Paris Saint-Germain,83


1、节点 - 足球运动员（按 ID）

2、边 - 如果他们为同一支球队效力

3、节点特征 - 足球运动员的位置、专长、控球、......

4、标签 - 足球运动员的总体评分（节点级回归任务）

节点通常很容易识别——在这里我们有 ID。 如果您没有唯一标识符，则需要一个，因为需要知道在哪些节点之间存在连接奥。

需要确保每个运动员的ID没有重复，这非常重要奥，请注意，我们计划在这里构建一个单一的图， 如果单个节点 ID 在咱们的数据集中出现不止一次，则有不同的选项，但这里为了简化，我们只考虑构建单一的图。

In [8]:
# Make sure that we have no duplicate nodes
max(fifa_df["int_player_id"].value_counts())

1

提取节点特征

In [9]:
# Sort to define the order of nodes
sorted_df = fifa_df.sort_values(by="int_player_id")
# Select node features
node_features = sorted_df[["str_positions", "int_long_passing", "int_ball_control", "int_dribbling"]]
# Convert non-numeric columns
pd.set_option('mode.chained_assignment', None)
positions = node_features["str_positions"].str.split(",", expand=True)
node_features["first_position"] = positions[0]
# One-hot encoding
node_features = pd.concat([node_features, pd.get_dummies(node_features["first_position"])], axis=1, join='inner')
node_features.drop(["str_positions", "first_position"], axis=1, inplace=True)
node_features.head() 

,int_long_passing,int_ball_control,int_dribbling,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,91,96,96,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,77,92,88,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,40,30,12,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,93,92,88,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,81,95,95,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [10]:
# Convert to numpy
x = node_features.to_numpy()
x.shape # [num_nodes x num_features]

(18767, 18)

这里，因为对应的是节点预测任务，我们将标签简单的设置为每个玩家的评分，也就是int_overall，这里节点的特征是比较多的，有些更简单的情况，你的标签就是你节点的特征

In [11]:
# Sort to define the order of nodes
sorted_df = fifa_df.sort_values(by="int_player_id")
# Select node features
labels = sorted_df[["int_overall"]]
labels.head()
# Convert to numpy
y = labels.to_numpy()
y.shape # [num_nodes, 1] --> node regression

(18767, 1)

这可能是大多数数据集最棘手的部分。 我们需要考虑一种合理的方式来连接您的节点。 我自己是做交通的，交通数据集通常比较容易确定边，例如站点之间是否直接相连，依据实际的地理位置等等。如前所述，我们将在这里使用团队分配。

我们现在需要找到分配给同一个团队的球员对。 让我们首先检查一下我们每支球队有多少球员。

In [12]:
# Remap player IDs
fifa_df["int_player_id"] = fifa_df.reset_index().index
# 确定每支队伍有多少球员要建立连接
fifa_df["str_team_name"].value_counts()

str_team_name
Everton                     36
TSG 1899 Hoffenheim         34
Borussia Mönchengladbach    34
Wolverhampton Wanderers     34
Brighton & Hove Albion      34
                            ..
Central Coast Mariners      19
Wellington Phoenix          19
Adelaide United             19
Brisbane Roar               19
Al Adalah                   19
Name: count, Length: 681, dtype: int64

我们使用列 int_player_id 作为边的索引。 例如，如果边索引中有 [0, 1]，则表示第一个和第二个节点（关于先前定义的节点特征矩阵）是相连的。

In [13]:
import itertools
import numpy as np

teams = fifa_df["str_team_name"].unique()
all_edges = np.array([], dtype=np.int32).reshape((0, 2))
for team in teams:
    team_df = fifa_df[fifa_df["str_team_name"] == team]
    players = team_df["int_player_id"].values
    # Build all combinations, as all players are connected
    permutations = list(itertools.combinations(players, 2))
    edges_source = [e[0] for e in permutations]
    edges_target = [e[1] for e in permutations]
    team_edges = np.column_stack([edges_source, edges_target])
    all_edges = np.vstack([all_edges, team_edges])
# Convert to Pytorch Geometric format
edge_index = all_edges.transpose()
edge_index # [2, num_edges]
#通常选择这种 COO 格式，因为它比 NxN 邻接矩阵更有效。

array([[    0,     0,     0, ..., 18698, 18698, 18728],
       [    8,    38,    39, ..., 18728, 18733, 18733]], dtype=int64)

构建数据集

In [14]:
from torch_geometric.data import Data
data = Data(x=x, edge_index=edge_index, y=y)
from torch_geometric.loader import DataLoader
data_list = [Data(...), ..., Data(...)]
loader = DataLoader(data_list, batch_size=32)

